In [17]:
import pandas as pd
import openai
import random
import ast
import os
import re
safe_globals = {"__builtins__": {}}
import pandas as pd
import random
import requests
import json
import transformers
import torch

In [2]:
openai.api_key = ""

In [4]:
data = pd.read_csv("../../Data/commonsense_qa.csv")
result = {}
error_list = []
user_temperature = 0.1

In [5]:
answer_options_formatted = []
for i in data['choices']:
    input_cleaned = re.sub(r"array\(\s*\[(.*?)\]\s*,\s*dtype=object\)", r"[\1]", i, flags=re.DOTALL)
    try:
        extracted_dict = eval(input_cleaned, safe_globals)
        labels = extracted_dict.get('label', [])
        texts = extracted_dict.get('text', [])
        if len(labels) == len(texts):
            formatted_output = '\n'.join([f"{label}: {text}" for label, text in zip(labels, texts)])
            answer_options_formatted.append(formatted_output)
        else:
            print(f"Skipping row due to mismatched lengths or missing data: {input_cleaned}")
    
    except (SyntaxError, NameError, TypeError) as e:
        print(f"Error parsing row: {input_cleaned}\nError details: {e}")

In [6]:
data['answer_options_formatted'] = answer_options_formatted

In [7]:
data['answer'] = data['answerKey'].replace({'A': 0, 'B': 1, 'C': 2, 'D': 3})

In [8]:
choice_list = list()
for i in data['answer_options_formatted']:
    output_list = [line.split(': ', 1)[1] for line in i.splitlines()]
    choice_list.append(output_list)

In [9]:
data['choices'] = choice_list

In [10]:
l = data['choices'].tolist()
data['choices_copy'] = l

In [11]:
def mask_choice(choice_list, correct_index):
    choice_list_copy = choice_list[:]
    masked_options = []
    incorrect_indices = [i for i in range(len(choice_list_copy)) if i != correct_index]
    
    if incorrect_indices:
        mask_index_1 = random.choice(incorrect_indices)
        masked_options.append(choice_list_copy[mask_index_1])  
        choice_list_copy[mask_index_1] = ""  
        incorrect_indices.remove(mask_index_1)  
    
    return choice_list_copy, masked_options

In [12]:
data[['choices_copy', 'masked_options']] = data.apply(
    lambda row: pd.Series(mask_choice(row['choices_copy'], row['answer'])), axis=1
)

In [13]:
def format_question_answer(row):
    question = f"Question: {row['question']}\nAnswer:\n"
    
    choices = row['choices_copy']
    options_text = [
        f"A: {str(choices[0]) if len(choices) > 0 else ''}",
        f"B: {str(choices[1]) if len(choices) > 1 else ''}",
        f"C: {str(choices[2]) if len(choices) > 2 else ''}",
        f"D: {str(choices[3]) if len(choices) > 3 else ''}",
        f"E: {str(choices[4]) if len(choices) > 4 else ''}"
    ]
    answer_options = "\n".join(options_text)
    
    return question + answer_options

In [14]:
data['question_answer'] = data.apply(format_question_answer, axis=1)

In [18]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
result = {}
for index, row in data[:].iterrows():
    
    prompt = f"""
        Q:Complete the missing option in one word: [
        Q: Which of the following countries generated the most total energy from solar sources in 2019?
        Answer:
        A: China
        B: United States
        C: 
        D: Japan
        ]. 
        Reply with ONLY ONE word the missing option which is enclosed in **[ANSWER]**, not with the entire sentence.
        A: **Germany**
        Similarly, complete the missing option in one word: [{row['question_answer']}]. Reply with ONLY ONE word which is enclosed in **[ANSWER]**, not with the entire sentence. Only answer the option asked, do not reply with any other information. Do not assume anything else is given to you other than the question and the options. Take the question and options given, and help deduce the missing option."""
    response = pipeline(prompt, max_new_tokens=10, temperature=0.5, top_p=0.9)
    response_text = response[0]['generated_text'][len(prompt):].strip()
    # answer = generated_text[len(prompt):].strip()
    result[row['question']] = response_text

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id

In [20]:
import re

def extract_final_answer(response_text):
    match = re.search(r"\*\*\s*\[?([^*\[\]]+)\]?\s*\*\*", response_text)
    if match:
        candidate = match.group(1).strip()
        if candidate and candidate != "[ANSWER]":  
            return candidate
    second_answer_match = re.search(r"A:\s*\[?([^\[\]]+)\]?", response_text)
    if second_answer_match:
        candidate = second_answer_match.group(1).strip()
        if candidate and candidate != "[ANSWER]":
            return candidate
    return "Invalid response"

processed_output = dict()

for key, val in result.items():
    processed_output[key] = extract_final_answer(val).lower()

In [24]:
acc = 0
for index, row in data.iterrows():
    if row['masked_options'] and len(row['masked_options']) > 0:
        first_option = row['masked_options'][0].lower()
        comparison = processed_output[row['question']].lower()
        if first_option == comparison:
            acc += 1
            break

In [27]:
print(f"The accuracy of Llama-3.2-3B-Instruct to predict the missing option is: {round(acc*100.0/len(result),2)}%")

The accuracy of Llama-3.2-3B-Instruct to predict the missing option is: 1.0%
